In [1]:
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [26]:
import os
myDf=spark.read.text(os.path.join("../data", "전문.txt")) 

# 문제 1) DataFrame을 생성

In [33]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

get_len = udf(lambda x: True if len(x) > 0 else False, BooleanType())
myDf = myDf.withColumn("txt_len", get_len(myDf.value))

In [34]:
myDf = myDf.filter(myDf.txt_len == True).drop('txt_len')
myDf.show()

+----------------------------------+
|                             value|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|         자랑스러운 경찰관 여러분,|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
| 정부는 누구도 법 위에 군림하지...|
| 경찰헌장은 따뜻한 경찰, 의로운...|
|행사를 준비하신 민갑룡 경찰청장...|
+----------------------------------+
only showing top 20 rows



# 문제 2) 단어로 분리해서, 출력

In [32]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="value", outputCol="words")

tokDf = tokenizer.transform(myDf)
tokDf.show()

+----------------------------------+--------------------------------+
|                             value|                           words|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...|
|         자랑스러운 경찰관 여러분,|   [자랑스러운, 경찰관, 여러분,]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|
|    대한민국 경찰은 1945년 광복...|    [대한민국, 경찰은, 1945년...|
| 임시정부 초대 경무국장 백범 김...|  [임시정부, 초대, 경무국장, ...|
|    광복 이후 6·25전쟁에서도 경...|    [광복, 이후, 6·25전쟁에서...|
|그러나 잘못도 없지는 않았습니다...|  [그러나, 잘못도, 없지는, 않...|
| 지금 경찰은 과거를 돌아보며 국...|  [지금, 경찰은, 과거를, 돌아...|
| 검경 수사권 조정과 자치경찰제 ...|  [검경, 수사권, 조정과, 자치...|
|정부는 경찰의 근무여건을 개선하...| [정

# 문제 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [63]:
txt_data = spark.sparkContext.textFile(os.path.join('../data','전문.txt'))
txt_data = txt_data.flatMap(lambda x : x.split()).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).map(lambda x : (x[1],x[0])).filter(lambda x : x[0]==1).collect()

stop_ls = []
for i in txt_data:
    stop_ls.append(i[1])

In [64]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords(stop_ls)

StopWordsRemover_94b859029ee5

In [65]:
for e in stop.getStopWords():
    print (e, end="/")

존경하는/국민/일흔네/돌/날’입니다./밤낮없이/애쓰시는/전국의/15만/전몰·순직/경찰관들의/고귀한/희생에/경의를/표합니다./유가족/위로의/마음을/전합니다./오늘/홍조근정훈장을/중앙경찰학교장/이은정/치안감님,/근정포장을/광주남부경찰서/김동현/경감님을/수상자/각별한/축하와/영웅으로/추서되신/차일혁,/최중락님께/사랑을/전해드립니다./사랑하는/여러분의/헌신적/노력으로/치안은/참/좋아졌습니다./지난해/범죄/발생은/2015년에/비해/15.1%/줄었습니다./같은/기간/교통사고/사망자는/18.2%/감소했습니다./치안의/개선은/체감으로/나타나고/올해/상반기/체감안전도는/74.5점으로/최고를/기록했습니다./범죄안전도는/처음으로/80점을/넘었습니다./한국을/찾는/관광객들도/치안을/좋게/평가합니다./한국의/무엇이/좋았느냐는/물음에/관광객들은/7년/연속으로/치안이/좋았다고/응답했습니다./개발도상국들은/모범으로/삼으려/올해는/날’에/맞춰/국제치안산업박람회와/서울국제경찰청장회의가/열립니다./치안/발전과/치안산업/발전이/세계에/널리/알려지게/될/자랑스러운/“나라와/위하여/충성”을/다한다는/다짐으로/시작합니다./헌장처럼/‘나라와/위한/충성’의/길을/걸으려/노력해/왔습니다./1945년/직후에/공식/탄생했습니다./뿌리는/임시정부에/닿아/초대/경무국장/백범/김구/선생과/나석주,/나창헌,/유상근/의사/등/앞장서서/일제와/싸웠습니다./일본/관헌에게/폭탄을/던지고,/밀정을/응징하며,/임정/요인들을/보호했습니다./이후/6·25전쟁에서도/국군과/피를/흘렸습니다./전쟁에서/1만여/명의/경찰관이/목숨을/잃었습니다./후로도/지키고/국가의/안보를/도왔습니다./헌신에/대해/거듭/감사의/말씀을/잘못도/없지는/않았습니다./한때/공권력을/무리하게/집행하며/인권을/훼손했습니다./부실하거나/불공정한/수사로/지탄을/받은/적도/무기력한/집행으로/국민께/걱정을/드리기도/했습니다./지금/과거를/돌아보며/국가에/충성하는/경찰로/거듭나려고/문재인/정부/들어/개혁위원회를/만들고/자체개혁에/나섰습니다./개혁을/국민은/큰/기대로/주목하고

# 문제 4) 불용어 제거하고, 출력

In [66]:
stopDf=stop.transform(tokDf)
stopDf.show()

+----------------------------------+--------------------------------+------------------------------+
|                             value|                           words|                       nostops|
+----------------------------------+--------------------------------+------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...| [여러분,, 경찰관, 여러분,,...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|[국민의, 안전을, 위해, 경찰...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[받으신, 받으신, 비롯한, 여...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|             [경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|                      [우리의]|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|[국민의, 있습니다., 국민의,...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...| [외국, 우리의, 가장, 외국,...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...| [‘경찰의, 함께, 우리의, 더...|
|         자랑스러운 경찰관 여러분,|   [자랑스러운, 경찰관, 여러분,]|             [경찰관, 여러분,]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|[경찰헌장은, 겨레를, 우리, 

# 문제 5) TF-IDF를 계산하고, 출력
# 문제 6) TF-IDF 컬럼을 features로 구성, 출력

In [68]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [74]:
for e in idfDf.select("idf").take(10):
    print(e)

Row(idf=SparseVector(262144, {118171: 2.9632, 142775: 1.9924, 160086: 1.2993}))
Row(idf=SparseVector(262144, {39431: 1.7047, 49855: 1.9924, 77757: 1.9924, 89318: 1.9924, 123553: 1.9924, 160086: 1.2993, 167255: 1.2993, 217323: 3.9849}))
Row(idf=SparseVector(262144, {39431: 1.7047, 49855: 1.9924, 82902: 1.9924, 84159: 1.9924, 118725: 4.7958, 167255: 1.2993, 217323: 1.9924, 251574: 1.9924}))
Row(idf=SparseVector(262144, {118171: 1.4816, 160086: 1.2993}))
Row(idf=SparseVector(262144, {256489: 1.7047}))
Row(idf=SparseVector(262144, {53553: 1.1451, 166402: 1.7047, 167255: 2.5986}))
Row(idf=SparseVector(262144, {23924: 1.9924, 39110: 3.9849, 67651: 4.7958, 69304: 1.9924, 256489: 3.4095}))
Row(idf=SparseVector(262144, {116270: 1.7047, 142775: 1.9924, 215575: 1.9924, 217049: 1.9924, 256489: 1.7047}))
Row(idf=SparseVector(262144, {118171: 1.4816, 160086: 1.2993}))
Row(idf=SparseVector(262144, {20610: 1.9924, 80957: 1.1451, 154116: 4.7958, 223474: 1.9924}))
